# DevMuT Artifact

Artifact of the paper "DevMuT: Testing Deep Learning Framework via Developer
Expertise-Based Mutation" from ASE 2024.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

### Lease creation

We'll create a short lease for one Nvidia Jetson Nano device.

In [ ]:
import chi
import os

chi.use_site("CHI@Edge")
chi.set("DevMuT_artifact", CHI-251412)

# get your username, just used to name leases something identifiable
username = os.environ.get("USER")

machine_name = "jetson-nano"

from chi import lease

# get dates for lease start and end
start, end = lease.lease_duration(days=1)

# make a unique name for the lease
lease_name = f"{username}-{machine_name}-{start}"

reservations = []
lease.add_device_reservation(reservations, count=1, machine_name=machine_name)
container_lease = lease.create_lease(lease_name, reservations)
lease_id = container_lease["id"]

print(f"created lease with name {lease_name} and uuid {lease_id}, waiting for it to start. This can take up to 60s.")
lease.wait_for_active(lease_id)
print("Done!")

### Container creation

Next, we are going to launch a container on our device, ensuring we set the additional `runtime` argument to `nvidia` to make sure that all nvidia compute modules are loaded and the nvidia cuda libraries are mounted onto the container. 

We will be using an nvidia provided PyTorch for Linux4Tegra image (`nvcr.io/nvidia/l4t-pytorch:r32.7.1-pth1.10-py3`) compatible with the current version of L4T installed on our nvidia jetson devices. PyTorch in itself requires a variety of CUDA libraries that are all mounted onto the container when `runtime="nvidia"` is specified during `container.create_container()` call.

***Note***: The container may fail to schedule sometimes due to intermittent failures with devices, this cell is idempotent and can be rerun if needed.

In [ ]:
from chi import container

try:
    container.destroy_container(f"tutorial-{machine_name}-nvidia-runtime".replace("_","-"))
except:
    pass

print("Requesting container ... This may take a while as the container image (800MB) is being downloaded")

# set a name for the container. Because CHI@Edge uses Kubernetes, ensure that underscores aren't in the name
container_name = f"tutorial-{machine_name}-nvidia-runtime".replace("_","-")

try:
    my_container = container.create_container(
        container_name, 
        image="nvcr.io/nvidia/l4t-pytorch:r32.7.1-pth1.10-py3",
        command=["/bin/bash", "-c", "while true; do sleep infinity; done"],
        workdir="/home",
        runtime="nvidia",
        reservation_id=lease.get_device_reservation(lease_id),
        platform_version=2,
    )
except RuntimeError as ex:
    print(ex)
    print(f"please stop and/or delete {container_name} and try again")
else:
    print(f"Successfully created container: {container_name}!")

## Run DevMuT

Now, we can finally run DevMuT. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [ ]:
# my_server.execute("git clone https://github.com/radhofan/DevMuT.git")
# container.upload(my_container.uuid, "simple_nn.py", "/home")
# print(container.execute(my_container.uuid, "ls -l /home")["output"])
print(container.execute(my_container.uuid, "git clone https://github.com/radhofan/DevMuT.git")["output"])

In [ ]:
# my_server.execute("chmod +x DevMuT/reproduce.sh")
# my_server.execute("bash DevMuT/reproduce.sh")
print(container.execute(my_container.uuid, "bash DevMuT/reproduce.sh")["output"])

In [ ]:
container.destroy_container(my_container.uuid)
lease.delete_lease(lease_id)